<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>modeling</a></span></li></ul></li></ul></div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('stopwords')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np 
import pandas as pd
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [3]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=None)
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


In [4]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label
y_test = test_label

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(11083,)
(7761,)
(11083,)
(7761,)


# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report
        
- Time complexity 
    - O(n^(dk+1)) where n is the number of observatons, d is the dimensionality, and k is k clusters

## modeling

In [6]:
# dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=3)
dtm_train, dtm_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=False, min_df=3)
# dtm_train, transform_mapper = dimension_reduction(dtm_train, out_dim=8000)
# dtm_test = transform_mapper.transform(dtm_test)
dtm_train = dtm_train.toarray()
dtm_test = dtm_test.toarray()

print('dtm_train.shape', dtm_train.shape)
print('dtm_test.shape', dtm_test.shape)
# not suitable for vectors that reduce dimension. tfidf information are distort
clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=100, metric='Cosine', repeats=5) 
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)

pred = list(map(lambda x: label_mapper[x], pred))
classification_report = evaluation_report(y_test, pred, labels=labels)

num of words: 38308
dtm_train.shape (11083, 38308)
dtm_test.shape (7761, 38308)
Group to label mapping: 
Group 0 <-> label 2
Group 1 <-> label 7
Group 2 <-> label 3
Group 3 <-> label 17
Group 4 <-> label 15
Group 5 <-> label 6
Group 6 <-> label 1
Group 7 <-> label 10
Group 8 <-> label 9
Group 9 <-> label 14
Group 10 <-> label 2
Group 11 <-> label 5
Group 12 <-> label 4
Group 13 <-> label 3
Group 14 <-> label 2
Group 15 <-> label 6
Group 16 <-> label 6
Group 17 <-> label 12
Group 18 <-> label 1
Group 19 <-> label 6
Group 20 <-> label 3
Group 21 <-> label 5
Group 22 <-> label 9
Group 23 <-> label 6
Group 24 <-> label 19
Group 25 <-> label 9
Group 26 <-> label 1
Group 27 <-> label 9
Group 28 <-> label 5
Group 29 <-> label 5
Group 30 <-> label 2
Group 31 <-> label 2
Group 32 <-> label 6
Group 33 <-> label 2
Group 34 <-> label 8
Group 35 <-> label 8
Group 36 <-> label 5
Group 37 <-> label 14
Group 38 <-> label 7
Group 39 <-> label 5
Group 40 <-> label 8
Group 41 <-> label 7
Group 42 <-> lab

KeyError: ignored

In [ ]:
# dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, min_df=7, max_df=0.01)
dtm_train, dtm_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, min_df=7, max_df=0.01)
dtm_train, transform_mapper = dimension_reduction(dtm_train, out_dim=500)
dtm_test = transform_mapper.transform(dtm_test)

print('dtm_train.shape', dtm_train.shape)
print('dtm_test.shape', dtm_test.shape)
clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=50, metric='L2', repeats=20)
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
print(metrics.classification_report(y_true = test_label, y_pred=pred))